In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import urllib.request

In [2]:
# Copied from anki connect site
def request(action: str, **params) -> dict:
    return {"action": action, "params": params, "version": 6}


# Copied from anki connect site
def invoke(action: str, **params):
    requestJson = json.dumps(request(action, **params)).encode("utf-8")
    response = json.load(
        urllib.request.urlopen(
            urllib.request.Request("http://127.0.0.1:8765", requestJson)
        )
    )
    if len(response) != 2:
        raise Exception("response has an unexpected number of fields")
    if "error" not in response:
        raise Exception("response is missing required error field")
    if "result" not in response:
        raise Exception("response is missing required result field")
    if response["error"] is not None:
        raise Exception(response["error"])
    return response["result"]

In [66]:
db_path: str = "data/clean/bir_warehouse.db"

year_num_options: list = [[4, 2015], [5, 2014]]

query_question_text: str = """
SELECT qn.question, ex.exam_type, yr.year_name, qo.option_text, qo.is_correct
FROM questions_options AS qo
JOIN questions AS qn ON qo.question_id = qn.id
JOIN year AS yr ON qn.exam_year = yr.id_year
JOIN exam AS ex ON qn.exam_subject = ex.id_type;
"""

conn = sqlite3.connect(db_path)

exams_df = pd.read_sql(query_question_text, conn)

conn.close()
exams_df["is_correct"] = exams_df["is_correct"].astype(bool)
exams_df["question_idx"] = exams_df["question"] + exams_df["exam_type"] + exams_df["year_name"]
exams_df["question"] = exams_df["question"].str.strip()
exams_df["year_name"] = exams_df["year_name"].astype(int)

exams_df.head()

,question,exam_type,year_name,option_text,is_correct,question_idx
0,1. La barrera hematoencefálica:,bir,2024,1. Es permeable a todas las sustancias present...,False,1. La barrera hematoencefálica:bir2024
1,1. La barrera hematoencefálica:,bir,2024,2. Es permeable al O2 y al CO2.,True,1. La barrera hematoencefálica:bir2024
2,1. La barrera hematoencefálica:,bir,2024,3. Es impermeable al etanol.,False,1. La barrera hematoencefálica:bir2024
3,1. La barrera hematoencefálica:,bir,2024,4. Es impermeable al agua.,False,1. La barrera hematoencefálica:bir2024
4,2. El espacio subaracnoideo se encuentra:,bir,2024,1. Entre la aracnoides y la duramadre.,False,2. El espacio subaracnoideo se encuentra:bir2024


In [67]:
print(len(exams_df['question_idx'].unique()))

5435


In [68]:
exams_df["option_num"] = exams_df.groupby(["question_idx", "exam_type", "year_name"]).cumcount() +1
exams_df.head()

,question,exam_type,year_name,option_text,is_correct,question_idx,option_num
0,1. La barrera hematoencefálica:,bir,2024,1. Es permeable a todas las sustancias present...,False,1. La barrera hematoencefálica:bir2024,1
1,1. La barrera hematoencefálica:,bir,2024,2. Es permeable al O2 y al CO2.,True,1. La barrera hematoencefálica:bir2024,2
2,1. La barrera hematoencefálica:,bir,2024,3. Es impermeable al etanol.,False,1. La barrera hematoencefálica:bir2024,3
3,1. La barrera hematoencefálica:,bir,2024,4. Es impermeable al agua.,False,1. La barrera hematoencefálica:bir2024,4
4,2. El espacio subaracnoideo se encuentra:,bir,2024,1. Entre la aracnoides y la duramadre.,False,2. El espacio subaracnoideo se encuentra:bir2024,1


In [69]:
exam_df_pv = exams_df.pivot_table(
    index= ["question", "exam_type", "year_name"],
    columns= "option_num",
    values="option_text",
    aggfunc= "first"
)
exam_df_pv.head()

,,option_num,1,2,3,4,5,6,7,8,9,10
question,exam_type,year_name,,,,,,,,,,
1. Cada vuelta de la hélice contiene 3.6 residuos de aminoácido.,qir,2019,2. Puede ser paralela o antiparalela.,3. Puede acomodar cualquier tipo de aminoácido...,"4. En esta estructura secundaria, las cadenas ...",81. ¿Cuál de las siguientes afirmaciones acerc...,NaN,NaN,NaN,NaN,NaN,NaN
1. Conduce a la síntesis mitocondrial de glucosa a partir de acetil-CoA en mamíferos.,qir,2019,2. Utiliza exactamente las mismas enzimas que ...,3. Consume una cantidad de energía metabólica ...,"4. Es de naturaleza catabólica, aunque genera ...",102. ¿Qué tipo de inhibidor enzimático AUMENTA...,NaN,NaN,NaN,NaN,NaN,NaN
1. Confiere a la membrana plasmática alta pe rmeabilidad al agua:,bir,2014,1. Acuaporinas.,1. Acuaporinas.,2. Canales iónicos.,3. La Na+/K+-ATPasa.,4. Intercambiador Cl/HCO3.,5. Su composición lípidica.,NaN,NaN,NaN,NaN
1. Cósmido.,qir,2019,2. Virus.,3. Fago.,4. Operón.,"99. La fructosa 2,6-bisfosfato:",NaN,NaN,NaN,NaN,NaN,NaN
1. Dideoxinucleótidos.,qir,2019,2. Ribonucleótidos.,3. Desoxinucleótidos.,4. Nucleósidos.,96. ¿Qué es un plásmido?:,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
print(len(exams_df))

exams_df["correct_int"] = np.where(
    exams_df["is_correct"], 
    exams_df["option_text"].apply(lambda x: x.split(".", 1)[0]),
    np.nan)

exams_df.head()

24286


,question,exam_type,year_name,option_text,is_correct,correct_int
0,1. La barrera hematoencefálica:,bir,2024,1. Es permeable a todas las sustancias present...,False,NaN
1,1. La barrera hematoencefálica:,bir,2024,2. Es permeable al O2 y al CO2.,True,2
2,1. La barrera hematoencefálica:,bir,2024,3. Es impermeable al etanol.,False,NaN
3,1. La barrera hematoencefálica:,bir,2024,4. Es impermeable al agua.,False,NaN
4,2. El espacio subaracnoideo se encuentra:,bir,2024,1. Entre la aracnoides y la duramadre.,False,NaN


In [ ]:
exams_questions = exams_df.iloc[:, 0:4]
exams_questions["option_id"] = exams_questions["option_text"].apply(lambda x: x.split(".", 1)[0])
exams_questions["num_q"] = exams_questions["question"].apply(lambda x: x.split(".", 1)[0])
exams_questions.head()

,question,exam_type,year_name,option_text,option_id,num_q
0,1. La barrera hematoencefálica:,bir,2024,1. Es permeable a todas las sustancias present...,1,1
1,1. La barrera hematoencefálica:,bir,2024,2. Es permeable al O2 y al CO2.,2,1
2,1. La barrera hematoencefálica:,bir,2024,3. Es impermeable al etanol.,3,1
3,1. La barrera hematoencefálica:,bir,2024,4. Es impermeable al agua.,4,1
4,2. El espacio subaracnoideo se encuentra:,bir,2024,1. Entre la aracnoides y la duramadre.,1,2


In [48]:
exams_questions_piv = exams_questions.pivot(
    index= ["question", "exam_type", "year_name"],
    columns= "option_id",
    values= "option_text"
)
exams_questions_piv.head()

ValueError: Index contains duplicate entries, cannot reshape

In [44]:
exams_options = exams_df[["year_name", "correct_int"]]
exams_options.head()

,year_name,correct_int
0,2024,NaN
1,2024,2
2,2024,NaN
3,2024,NaN
4,2024,NaN
